# Building a Convolutional Neural Network for Sea Ice/Water Classification

# Importing training/validation dataset


In [ ]:
import numpy as np

In [ ]:
Xice = np.load(r"../../data/TrainingDataS0_ice_rev.npy")
Xwater = np.load(r"../../data/TrainingDataS0_water_rev.npy")

y_ice = np.zeros(len(Xice), dtype=np.int8)
y_water = np.ones(len(Xwater), dtype=np.int8)
y = np.concatenate((y_ice,y_water))

X = np.concatenate((Xice,Xwater))

del y_ice, y_water, Xice, Xwater

In [ ]:
#Shuffles the dataset to ensure random selection of training/validation samples
#rng_state ensure that the shuffling stays the same for both arrays
rng_state = np.random.get_state()
np.random.shuffle(X)
np.random.set_state(rng_state)
np.random.shuffle(y)

np.save(r"../../data/TrainingDataS0_shuffled.npy", X)
np.save(r"../../data/ClassDataS0_shuffled.npy", y)

In [ ]:
#Calculating normalization parameters
means = []
stds = []

for i in np.arange(X.shape[3]):
    means.append(np.mean(X[:,:,:,i].astype(np.float64)))
    stds.append(np.std(X[:,:,:,i].astype(np.float64)))

In [ ]:
import pandas as pd
df = pd.DataFrame({'means':means, 'stds':stds}, 
                  index=['HH','HV','Angle','Noise'])
df.to_pickle(r"../../data/means_stds_S0_cnn.pkl")
print(means,stds)

In [ ]:
#Nomalizing dataset by mean and standard deviation to improve CNN accuracy
for i in np.arange(X.shape[3]):
    X[:,:,:,i] = ((X[:,:,:,i].astype(np.float64) - df.means.iloc[i]) / df.stds.iloc[i]).astype(np.float64)

# Building the CNN

In [ ]:
# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Initialising the CNN
classifierhh = Sequential()

In [ ]:
#Initializing the convolution layer imput parameters
convlayers = 128 # Number of filters
convwindsize1 = 5 # Window Size (horizontal)
convwindsize2 = 5 # Window Size (vertical)
imheight = 20 # Height of input image
imwidth = 20 # Width of imput image
imbands = 4  # Number of image bands

In [ ]:
# Step 1 - Convolution
classifierhh.add(Conv2D(convlayers, (convwindsize1, convwindsize1), strides =(1,1),
                        input_shape = (imwidth, imheight, imbands), activation = 'relu'))

In [ ]:
# Step 2 - Pooling
poolsize = 2 # Window Size to extract the maximum value from (MaxPooling)
classifierhh.add(MaxPooling2D(pool_size = (poolsize, poolsize)))

In [ ]:
# Adding a second convolution/pool layer
classifierhh.add(Conv2D(convlayers, (convwindsize2, convwindsize2), strides = (1,1), activation = 'relu'))
classifierhh.add(MaxPooling2D(pool_size = (poolsize, poolsize)))

In [ ]:
# Step 3 - Flattening
classifierhh.add(Flatten())

In [ ]:
# Step 4 - Full connection
outdim1 = 1024 #Number of nodes in the 1st fully connected layer
classifierhh.add(Dense(units = outdim1, activation = 'relu'))
# Randomly drop 10% of the nodes during training to avoid overfitting of specific nodes
classifierhh.add(Dropout(rate = 0.1))
outdim2 = 128 # Number of nodes in the 2nd fully connected layer
classifierhh.add(Dense(units = outdim2, activation = 'relu'))
classifierhh.add(Dropout(rate = 0.1))
outdim3 = 1 # Number of nodes in 3rd fully connected layer. Since we only have a binary problem (ice/water),
            # only 1 dimension is needed
classifierhh.add(Dense(units = outdim3, activation = 'sigmoid'))

In [ ]:
# Compiling the CNN
classifierhh.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Generating training stopage conditions to avoid overfitting
reduce_lr = ReduceLROnPlateau(monitor='accuracy', min_delta = 0.03, factor=0.7, patience=5, verbose = 1)
early_stop = EarlyStopping(monitor='val_accuracy', min_delta = 1E-4, patience = 10, verbose = 1)
checkpointer = ModelCheckpoint(filepath = r'../../data/SeaIceCNN_CheckPoint.h5', monitor = 'accuracy',
                               verbose=1,
                               save_best_only=True, save_weights_only = True)

In [ ]:
#Train/Validation the CNN
history = classifierhh.fit(x = X, y = y, epochs = 100, callbacks =[reduce_lr, early_stop], validation_split = 0.3)

Train on 8924787 samples, validate on 3824910 samples <br>
Epoch 1/100<br> 
8924787/8924787 [==============================] - 1618s 181us/sample - loss: 0.2751 - accuracy: 0.8723 - val_loss: 0.2558 - val_accuracy: 0.8814<br>
Epoch 2/100<br>
8924787/8924787 [==============================] - 1628s 182us/sample - loss: 0.2514 - accuracy: 0.8832 - val_loss: 0.2459 - val_accuracy: 0.8859<br>
Epoch 3/100<br>
8924787/8924787 [==============================] - 1609s 180us/sample - loss: 0.2439 - accuracy: 0.8869 - val_loss: 0.2410 - val_accuracy: 0.8886<br>
Epoch 4/100<br>
8924787/8924787 [==============================] - 1605s 180us/sample - loss: 0.2394 - accuracy: 0.8891 - val_loss: 0.2386 - val_accuracy: 0.8898<br>
Epoch 5/100<br>
8924787/8924787 [==============================] - 1597s 179us/sample - loss: 0.2361 - accuracy: 0.8907 - val_loss: 0.2340 - val_accuracy: 0.8919<br>
Epoch 6/100<br>
8924576/8924787 [============================>.] - ETA: 0s - loss: 0.2335 - accuracy: 0.8921<br>
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.006999999843537807.<br>
8924787/8924787 [==============================] - 1589s 178us/sample - loss: 0.2335 - accuracy: 0.8921 - val_loss: 0.2325 - val_accuracy: 0.8920<br>
Epoch 7/100<br>
8924787/8924787 [==============================] - 1593s 179us/sample - loss: 0.2291 - accuracy: 0.8942 - val_loss: 0.2303 - val_accuracy: 0.8928<br>
Epoch 8/100<br>
8924787/8924787 [==============================] - 1620s 181us/sample - loss: 0.2275 - accuracy: 0.8949 - val_loss: 0.2286 - val_accuracy: 0.8944<br>
Epoch 9/100<br>
8924787/8924787 [==============================] - 1601s 179us/sample - loss: 0.2262 - accuracy: 0.8956 - val_loss: 0.2305 - val_accuracy: 0.8936<br>
Epoch 10/100<br>
8924787/8924787 [==============================] - 1584s 178us/sample - loss: 0.2249 - accuracy: 0.8962 - val_loss: 0.2272 - val_accuracy: 0.8950<br>
Epoch 11/100<br>
8924768/8924787 [============================>.] - ETA: 0s - loss: 0.2237 - accuracy: 0.8968<br>
Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.004899999825283885.<br>
8924787/8924787 [==============================] - 1608s 180us/sample - loss: 0.2237 - accuracy: 0.8968 - val_loss: 0.2275 - val_accuracy: 0.8946<br>
Epoch 12/100<br>
8924787/8924787 [==============================] - 1604s 180us/sample - loss: 0.2205 - accuracy: 0.8983 - val_loss: 0.2239 - val_accuracy: 0.8966<br>
Epoch 13/100<br>
8924787/8924787 [==============================] - 1583s 177us/sample - loss: 0.2195 - accuracy: 0.8988 - val_loss: 0.2248 - val_accuracy: 0.8959<br>
Epoch 14/100<br>
8924787/8924787 [==============================] - 1587s 178us/sample - loss: 0.2187 - accuracy: 0.8992 - val_loss: 0.2227 - val_accuracy: 0.8973<br>
Epoch 15/100<br>
8924787/8924787 [==============================] - 1576s 177us/sample - loss: 0.2179 - accuracy: 0.8996 - val_loss: 0.2233 - val_accuracy: 0.8971<br>
Epoch 16/100<br>
8924608/8924787 [============================>.] - ETA: 0s - loss: 0.2171 - accuracy: 0.8999<br>
Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0034300000406801696.<br>
8924787/8924787 [==============================] - 1589s 178us/sample - loss: 0.2171 - accuracy: 0.8999 - val_loss: 0.2226 - val_accuracy: 0.8970<br>
Epoch 17/100<br>
8924787/8924787 [==============================] - 1598s 179us/sample - loss: 0.2146 - accuracy: 0.9013 - val_loss: 0.2207 - val_accuracy: 0.8983<br>
Epoch 18/100<br>
8924787/8924787 [==============================] - 1598s 179us/sample - loss: 0.2138 - accuracy: 0.9016 - val_loss: 0.2203 - val_accuracy: 0.8985<br>
Epoch 19/100<br>
8924787/8924787 [==============================] - 1589s 178us/sample - loss: 0.2133 - accuracy: 0.9017 - val_loss: 0.2201 - val_accuracy: 0.8986<br>
Epoch 20/100<br>
8924787/8924787 [==============================] - 1583s 177us/sample - loss: 0.2127 - accuracy: 0.9020 - val_loss: 0.2205 - val_accuracy: 0.8984<br>
Epoch 21/100<br>
8924736/8924787 [============================>.] - ETA: 0s - loss: 0.2121 - accuracy: 0.9022<br>
Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.002401000028476119.<br>
8924787/8924787 [==============================] - 1602s 179us/sample - loss: 0.2121 - accuracy: 0.9022 - val_loss: 0.2194 - val_accuracy: 0.8988<br>
Epoch 22/100<br>
8924787/8924787 [==============================] - 1606s 180us/sample - loss: 0.2099 - accuracy: 0.9033 - val_loss: 0.2187 - val_accuracy: 0.8993<br>
Epoch 23/100<br>
8924787/8924787 [==============================] - 1578s 177us/sample - loss: 0.2094 - accuracy: 0.9036 - val_loss: 0.2190 - val_accuracy: 0.8990<br>
Epoch 24/100<br>
8924787/8924787 [==============================] - 1574s 176us/sample - loss: 0.2090 - accuracy: 0.9038 - val_loss: 0.2195 - val_accuracy: 0.8987<br>
Epoch 25/100<br>
8924787/8924787 [==============================] - 1587s 178us/sample - loss: 0.2085 - accuracy: 0.9041 - val_loss: 0.2187 - val_accuracy: 0.8992<br>
Epoch 26/100<br>
8924787/8924787 [==============================] - 1613s 181us/sample - loss: 0.2080 - accuracy: 0.9043 - val_loss: 0.2185 - val_accuracy: 0.8993<br>
Epoch 27/100<br>
8924736/8924787 [============================>.] - ETA: 0s - loss: 0.2075 - accuracy: 0.9046<br>
Epoch 00027: ReduceLROnPlateau reducing learning rate to 0.0016807000851258634.<br>
8924787/8924787 [==============================] - 1604s 180us/sample - loss: 0.2075 - accuracy: 0.9046 - val_loss: 0.2190 - val_accuracy: 0.8993<br>
Epoch 28/100<br>
8924787/8924787 [==============================] - 1581s 177us/sample - loss: 0.2057 - accuracy: 0.9054 - val_loss: 0.2195 - val_accuracy: 0.8985<br>
Epoch 29/100<br>
8924787/8924787 [==============================] - 1580s 177us/sample - loss: 0.2052 - accuracy: 0.9056 - val_loss: 0.2177 - val_accuracy: 0.8998<br>
Epoch 30/100<br>
8924787/8924787 [==============================] - 1605s 180us/sample - loss: 0.2049 - accuracy: 0.9058 - val_loss: 0.2187 - val_accuracy: 0.8996<br>
Epoch 31/100<br>
8924787/8924787 [==============================] - 1610s 180us/sample - loss: 0.2044 - accuracy: 0.9060 - val_loss: 0.2173 - val_accuracy: 0.8999<br>
Epoch 32/100<br>
8924544/8924787 [============================>.] - ETA: 0s - loss: 0.2040 - accuracy: 0.9062<br>
Epoch 00032: ReduceLROnPlateau reducing learning rate to 0.0011764900758862494.<br>
8924787/8924787 [==============================] - 1594s 179us/sample - loss: 0.2040 - accuracy: 0.9062 - val_loss: 0.2173 - val_accuracy: 0.9001<br>
Epoch 33/100<br>
8924787/8924787 [==============================] - 1582s 177us/sample - loss: 0.2025 - accuracy: 0.9069 - val_loss: 0.2176 - val_accuracy: 0.8999<br>
Epoch 34/100<br>
8924787/8924787 [==============================] - 1606s 180us/sample - loss: 0.2021 - accuracy: 0.9072 - val_loss: 0.2173 - val_accuracy: 0.9001<br>
Epoch 35/100<br>
8924787/8924787 [==============================] - 1620s 182us/sample - loss: 0.2018 - accuracy: 0.9073 - val_loss: 0.2177 - val_accuracy: 0.8999<br>
Epoch 36/100<br>
8924787/8924787 [==============================] - 1614s 181us/sample - loss: 0.2015 - accuracy: 0.9074 - val_loss: 0.2175 - val_accuracy: 0.9000<br>
Epoch 37/100<br>
8924608/8924787 [============================>.] - ETA: 0s - loss: 0.2012 - accuracy: 0.9075<br>
Epoch 00037: ReduceLROnPlateau reducing learning rate to 0.0008235430694185197.<br>
8924787/8924787 [==============================] - 1577s 177us/sample - loss: 0.2012 - accuracy: 0.9075 - val_loss: 0.2172 - val_accuracy: 0.9002<br>
Epoch 38/100<br>
8924787/8924787 [==============================] - 1585s 178us/sample - loss: 0.2000 - accuracy: 0.9081 - val_loss: 0.2172 - val_accuracy: 0.9002<br>
Epoch 39/100<br>
8924787/8924787 [==============================] - 1607s 180us/sample - loss: 0.1996 - accuracy: 0.9083 - val_loss: 0.2172 - val_accuracy: 0.9003<br>
Epoch 40/100<br>
8924787/8924787 [==============================] - 1592s 178us/sample - loss: 0.1994 - accuracy: 0.9085 - val_loss: 0.2173 - val_accuracy: 0.9003<br>
Epoch 41/100<br>
8924787/8924787 [==============================] - 1586s 178us/sample - loss: 0.1992 - accuracy: 0.9085 - val_loss: 0.2174 - val_accuracy: 0.9000<br>
Epoch 42/100<br>
8924768/8924787 [============================>.] - ETA: 0s - loss: 0.1990 - accuracy: 0.9087<br>
Epoch 00042: ReduceLROnPlateau reducing learning rate to 0.0005764801404438912.<br>
8924787/8924787 [==============================] - 1601s 179us/sample - loss: 0.1990 - accuracy: 0.9087 - val_loss: 0.2171 - val_accuracy: 0.9001<br>
Epoch 43/100<br>
8924787/8924787 [==============================] - 1603s 180us/sample - loss: 0.1979 - accuracy: 0.9091 - val_loss: 0.2169 - val_accuracy: 0.9003<br>
Epoch 44/100<br>
8924787/8924787 [==============================] - 1613s 181us/sample - loss: 0.1977 - accuracy: 0.9092 - val_loss: 0.2173 - val_accuracy: 0.9003<br>
Epoch 45/100<br>
8924787/8924787 [==============================] - 1616s 181us/sample - loss: 0.1975 - accuracy: 0.9094 - val_loss: 0.2170 - val_accuracy: 0.9002<br>
Epoch 46/100<br>
8924787/8924787 [==============================] - 1580s 177us/sample - loss: 0.1973 - accuracy: 0.9094 - val_loss: 0.2169 - val_accuracy: 0.9005<br>
Epoch 47/100<br>
8924672/8924787 [============================>.] - ETA: 0s - loss: 0.1971 - accuracy: 0.9095<br>
Epoch 00047: ReduceLROnPlateau reducing learning rate to 0.0004035360820125788.<br>
8924787/8924787 [==============================] - 1582s 177us/sample - loss: 0.1971 - accuracy: 0.9095 - val_loss: 0.2174 - val_accuracy: 0.9003<br>
Epoch 48/100<br>
8924787/8924787 [==============================] - 1604s 180us/sample - loss: 0.1964 - accuracy: 0.9099 - val_loss: 0.2170 - val_accuracy: 0.9005<br>
Epoch 49/100<br>
8924787/8924787 [==============================] - 1610s 180us/sample - loss: 0.1962 - accuracy: 0.9099 - val_loss: 0.2173 - val_accuracy: 0.9004<br>
Epoch 50/100<br>
8924787/8924787 [==============================] - 1592s 178us/sample - loss: 0.1960 - accuracy: 0.9101 - val_loss: 0.2176 - val_accuracy: 0.9002<br>
Epoch 51/100<br>
8924787/8924787 [==============================] - 1587s 178us/sample - loss: 0.1959 - accuracy: 0.9101 - val_loss: 0.2169 - val_accuracy: 0.9006<br>
Epoch 52/100<br>
8924576/8924787 [============================>.] - ETA: 0s - loss: 0.1958 - accuracy: 0.9102<br>
Epoch 00052: ReduceLROnPlateau reducing learning rate to 0.0002824752533342689.<br>
8924787/8924787 [==============================] - 1602s 180us/sample - loss: 0.1958 - accuracy: 0.9102 - val_loss: 0.2170 - val_accuracy: 0.9005<br>
Epoch 53/100<br>
8924787/8924787 [==============================] - 1617s 181us/sample - loss: 0.1952 - accuracy: 0.9104 - val_loss: 0.2170 - val_accuracy: 0.9005<br>
Epoch 54/100<br>
8924787/8924787 [==============================] - 1616s 181us/sample - loss: 0.1951 - accuracy: 0.9106 - val_loss: 0.2173 - val_accuracy: 0.9005<br>
Epoch 55/100<br>
8924787/8924787 [==============================] - 1590s 178us/sample - loss: 0.1950 - accuracy: 0.9105 - val_loss: 0.2171 - val_accuracy: 0.9003<br>
Epoch 56/100<br>
8924787/8924787 [==============================] - 1584s 177us/sample - loss: 0.1949 - accuracy: 0.9106 - val_loss: 0.2170 - val_accuracy: 0.9004<br>
Epoch 57/100<br>
8924608/8924787 [============================>.] - ETA: 0s - loss: 0.1947 - accuracy: 0.9107<br>
Epoch 00057: ReduceLROnPlateau reducing learning rate to 0.0001977326814085245.<br>
8924787/8924787 [==============================] - 1596s 179us/sample - loss: 0.1947 - accuracy: 0.9107 - val_loss: 0.2171 - val_accuracy: 0.9005<br>
Epoch 58/100<br>
8924787/8924787 [==============================] - 1611s 181us/sample - loss: 0.1943 - accuracy: 0.9109 - val_loss: 0.2172 - val_accuracy: 0.9005<br>
Epoch 59/100<br>
8924787/8924787 [==============================] - 1593s 178us/sample - loss: 0.1943 - accuracy: 0.9109 - val_loss: 0.2171 - val_accuracy: 0.9005<br>
Epoch 60/100<br>
8924787/8924787 [==============================] - 1582s 177us/sample - loss: 0.1942 - accuracy: 0.9109 - val_loss: 0.2175 - val_accuracy: 0.9005<br>
Epoch 61/100<br>
8924787/8924787 [==============================] - 1589s 178us/sample - loss: 0.1941 - accuracy: 0.9110 - val_loss: 0.2171 - val_accuracy: 0.9005<br>
Epoch 00061: early stopping

In [ ]:
#Save the CNN model to reload in the future and classify independent SAR scenes
classifierhh.save(r'../../data/SeaIceCNN.h5')

In [ ]:
#Save the history results of the training/validation
import pickle
with open(r'../../data/SeaIceCNN.pkl','wb') as sfile:
    pickle.dump(history.history, sfile)